# My little playground

Build a CNN to classify task

In [1]:
from pathlib import Path
from utils.loaders import load_data_multiple_subjects

from utils.transforms import * 
from utils.dataset import EEGDataset
from torch.utils.data import random_split, DataLoader
import torch.nn as nn

eeg_dir = Path('./EEGDataset')
subjects = ['sub-01']

In [2]:
transforms = Compose([
    ToTensor(),
    Resize(600),
])

data = EEGDataset(eeg_dir, subjects, transforms)
train, test = random_split(data, [0.8, 0.2])

# loaders
train_loader = DataLoader(train, batch_size=32)
test_loader  = DataLoader(test, batch_size=32)

Define convolutional model here:

In [10]:

class CNN(torch.nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.Conv1 = nn.Sequential(
            # temporal convolution
            nn.Conv2d(1, 64, kernel_size=(1,10), padding='same'),
            nn.ReLU(),

            # spatial convolution
            nn.Conv2d(64, 64, kernel_size=(4,1), padding='same'),
            nn.ReLU(),

            # pooling
            nn.MaxPool2d((4,4))
        )
        self.Conv2 = nn.Sequential(
            # temporal convolution
            nn.Conv2d(64, 128, kernel_size=(1,10), padding='same'),
            nn.ReLU(),

            # spatial convolution
            nn.Conv2d(128, 128, kernel_size=(4,1), padding='same'),
            nn.ReLU(),

            # pooling
            nn.MaxPool2d((2,3))
        )
        self.Conv3 = nn.Sequential(
            # temporal convolution
            nn.Conv2d(128, 256, kernel_size=(1,5), padding='same'),
            nn.ReLU(),

            # spatial convolution
            nn.Conv2d(256, 256, kernel_size=(2,1), padding='same'),
            nn.ReLU(),

            # pooling
            nn.MaxPool2d((2,2))
        )
        self.Conv4 = nn.Sequential( 
            # temporal convolution
            nn.Conv2d(256, 64, kernel_size=(1,5), padding='same'),
            nn.ReLU(),

            # spatial convolution
            nn.Conv2d(64, 64, kernel_size=(2,1), padding='same'),
            nn.ReLU() ,

            nn.MaxPool2d((2,5)),
        )

        self.lin1 = nn.Linear(4*5*64, 1024)
        self.lin2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = x[:,None,:,:].float()  # reshape with 1 channel

        # convolutional layers
        x = self.Conv1(x)
        x = self.Conv2(x)
        x = self.Conv3(x)
        x = self.Conv4(x)

        # mlp
        x = x.flatten(start_dim=1)
        x = self.lin1(x)
        x = self.lin2(x)

        return x
    
class MLP(torch.nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(128*600, 1000),
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(1000, 500),
            nn.ReLU(),
        )
        self.fc3 = nn.Sequential(
            nn.Linear(500, 100),
            nn.ReLU(),
        )
        self.fc4 = nn.Sequential(
            nn.Linear(100, 2),
        )

    def forward(self, x):
        x = x.float()
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)

        return x 

        


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
loss_fn = nn.CrossEntropyLoss()


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()                       # if not train, model won't compute gradients

    for batch_idx, batch in enumerate(dataloader):
        X = batch['eeg'].to(device)
        y = batch['label'].to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), batch_idx * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def valid(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()                    # set to eval to not compute gradient

    val_loss, correct = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            X = batch['eeg'].to(device)
            y = batch['label'].to(device)

            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {val_loss:>8f} \n")



Train the model by looping over <code>train()</code>.

In [17]:
lr = 2e-3
    
model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    valid(train_loader, model, loss_fn)

print('Done!')

Epoch 1
-------------------------------
loss: 0.692661  [    0/  471]
Test Error: 
 Accuracy: 49.5%, Avg loss: 0.695105 

Epoch 2
-------------------------------
loss: 0.687775  [    0/  471]
Test Error: 
 Accuracy: 49.5%, Avg loss: 0.693293 

Epoch 3
-------------------------------
loss: 0.691845  [    0/  471]
Test Error: 
 Accuracy: 49.5%, Avg loss: 0.693166 

Epoch 4
-------------------------------
loss: 0.692890  [    0/  471]
Test Error: 
 Accuracy: 50.5%, Avg loss: 0.693140 

Epoch 5
-------------------------------
loss: 0.695134  [    0/  471]
Test Error: 
 Accuracy: 50.5%, Avg loss: 0.693121 

Epoch 6
-------------------------------
loss: 0.694637  [    0/  471]
Test Error: 
 Accuracy: 50.5%, Avg loss: 0.693116 

Epoch 7
-------------------------------
loss: 0.694237  [    0/  471]
Test Error: 
 Accuracy: 50.5%, Avg loss: 0.693115 

Epoch 8
-------------------------------
loss: 0.694204  [    0/  471]
Test Error: 
 Accuracy: 50.5%, Avg loss: 0.693116 

Epoch 9
----------------